# Homework 10
#### Course Notes
**Language Models:** https://github.com/rjenki/BIOS512/tree/main/lecture17  
**Unix:** https://github.com/rjenki/BIOS512/tree/main/lecture18  
**Docker:** https://github.com/rjenki/BIOS512/tree/main/lecture19

## Question 1
#### Make a language model that uses ngrams and allows the user to specify start words, but uses a random start if one is not specified.

#### a) Make a function to tokenize the text.

In [1]:
tokenize <- function(text) {
  text <- tolower(text)
  unlist(regmatches(text, gregexpr("[a-z']+|[.?!]", text, perl = TRUE)))
}


#### b) Make a function generate keys for ngrams.

In [2]:
.join_key  <- function(x) paste(x, collapse = "\x1F")
.split_key <- function(k) strsplit(k, "\x1F", fixed = TRUE)[[1]]

ngram_pairs <- function(tokens, n = 3) {
  if (n < 2) stop("n must be >= 2")
  if (length(tokens) < n)
    return(data.frame(key = character(0), next_token = character(0), stringsAsFactors = FALSE))
  idx  <- seq_len(length(tokens) - n + 1)
  keys <- vapply(idx, function(i) .join_key(tokens[i:(i + n - 2)]), character(1))
  nexts <- tokens[n:length(tokens)]
  data.frame(key = keys, next_token = nexts, stringsAsFactors = FALSE)
}


#### c) Make a function to build an ngram table.

In [3]:
build_ngram_table <- function(pairs) {
  if (!nrow(pairs)) return(list())
  split(pairs$next_token, pairs$key)
}


#### d) Function to digest the text.

In [4]:
digest_text <- function(text, n = 3) {
  toks  <- tokenize(text)
  pairs <- ngram_pairs(toks, n)
  table <- build_ngram_table(pairs)
  list(table = table, n = n)
}


#### e) Function to digest the url.

In [5]:
digest_url <- function(u, n = 3) {
  text <- tryCatch({
    readr::read_file(u)
  }, error = function(e) {
    con <- url(u, open = "rb")
    on.exit(try(close(con), silent = TRUE), add = TRUE)
    paste(readLines(con, warn = FALSE, encoding = "UTF-8"), collapse = "\n")
  })
  text <- gsub("<[^>]+>", " ", text)
  digest_text(text, n)
}


#### f) Function that gives random start.

In [6]:
random_start <- function(table) {
  key <- sample(names(table), 1)
  .split_key(key)
}


#### g) Function to predict the next word.

In [7]:
predict_next_word <- function(table, history, n) {
  if (length(history) < (n - 1)) return(NA_character_)
  key <- .join_key(tail(history, n - 1))
  if (!key %in% names(table)) return(NA_character_)
  sample(table[[key]], 1)
}


#### h) Function that puts everything together. Specify that if the user does not give a start word, then the random start will be used.

In [8]:
.detokenize <- function(tokens) {
  if (!length(tokens)) return("")
  out <- tokens[1]
  if (length(tokens) > 1) {
    for (tok in tokens[-1]) {
      if (tok %in% c(".", "?", "!")) out <- paste0(out, tok) else out <- paste(out, tok)
    }
  }
  out
}

generate_text <- function(table, n, max_words = 50, start_words = NULL) {
  if (is.null(start_words) || length(start_words) < (n - 1) ||
      !.join_key(tail(start_words, n - 1)) %in% names(table)) {
    history <- random_start(table)
  } else {
    history <- tail(start_words, n - 1)
  }
  generated <- history
  for (i in seq_len(max_words)) {
    nxt <- predict_next_word(table, history, n)
    if (is.na(nxt)) break
    generated <- c(generated, nxt)
    history <- tail(generated, n - 1)
  }
  .detokenize(generated)
}


## Question 2
#### For this question, set `seed=2025`.
#### a) Test your model using a text file of [Grimm's Fairy Tails](https://www.gutenberg.org/cache/epub/2591/pg2591.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 
#### ii) Using n=3, with no start word, with length=15.

In [9]:
set.seed(2025)
n  <- 3L
L  <- 15L  # number of tokens to generate
u_grimm  <- "https://www.gutenberg.org/cache/epub/2591/pg2591.txt"
u_armour <- "https://www.gutenberg.org/cache/epub/46342/pg46342.txt"

m_grimm  <- digest_url(u_grimm,  n = n)
m_armour <- digest_url(u_armour, n = n)
# i) n=3, start = "the king", length = 15
cat("a.i:\n")
out_ai  <- generate_text(m_grimm$table, m_grimm$n, max_words = L, start_words = c("the","king"))
cat(out_ai, "\n\n")

# ii) n=3, no start word, length = 15
cat("a.ii:\n")
out_aii <- generate_text(m_grimm$table, m_grimm$n, max_words = L)
cat(out_aii, "\n\n")


a.i:
the king said to the king and accused him of his friends and both studied law at 

a.ii:
horsemen would not rest until the liquid ran out of the wine ran out. then he 



#### b) Test your model using a text file of [Ancient Armour and Weapons in Europe](https://www.gutenberg.org/cache/epub/46342/pg46342.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 
#### ii) Using n=3, with no start word, with length=15.

In [10]:
# i) n=3, start = "the king", length = 15
cat("b.i:\n")
out_bi  <- generate_text(m_armour$table, m_armour$n, max_words = L, start_words = c("the","king"))
cat(out_bi, "\n\n")

# ii) n=3, no start word, length = 15
cat("b.ii:\n")
out_bii <- generate_text(m_armour$table, m_armour$n, max_words = L)
cat(out_bii, "\n\n")


b.i:
the king of england have been found with the description left us by sidonius as forming part 

b.ii:
this incident gives it in the pourpointed chausson worked in the last of the weapons disclosed by 



#### c) Explain in 1-2 sentences the difference in content generated from each source.

In [ ]:
# Grimm’s text produces narrative, character‑driven phrases (story events, dialogue, motifs), whereas the armour text yields technical, expository language about materials, weapons, and historical context; starting from “the king” fits Grimm naturally but in the armour corpus it is rarer and tends to drift quickly into descriptive prose.

## Question 3
#### a) What is a language learning model? 
#### b) Imagine the internet goes down and you can't run to your favorite language model for help. How do you run one locally?

In [ ]:
# A) A model that assigns probabilities to token sequences and predicts the next token from context; modern LLMs are neural (usually Transformers) trained via next‑token prediction on large corpora.
# B) Install OLLAMA, check the version with `ollama -v, Pull or install a model locally, Run the OLLAMA API server.

## Question 4
#### Explain what the following vocab words mean in the context of typing `mkdir project` into the command line. If the term doesn't apply to this command, give the definition and/or an example.
| Term | Meaning |  
|------|---------|
| **Shell** | The command interpreter (e.g., bash, zsh) that parses mkdir project, finds mkdir via PATH, and launches it with argument project. |
| **Terminal emulator** | The app/window you type into (e.g., Terminal, iTerm2); it hosts the shell and displays any output. |
| **Process** | The running instance of the mkdir program created by the shell to execute the command. |
| **Signal** | Async control message to a process; not used normally here. Example: Ctrl‑C sends SIGINT to abort mkdir. |
| **Standard input** | The input stream to the process; mkdir doesn’t read stdin (it’s ignored). |
| **Standard output** | The normal output stream; mkdir is silent on success (prints only with flags like -v); errors go to stderr. |
| **Command line argument** | The values after the command name; here project (and options like -p) are arguments passed to mkdir. |
| **The environment** | Inherited key–value vars. Example: PATH lets the shell locate mkdir; LANG/LC_* can affect message language. |

## Question 5
#### Consider the following command `find . -iname "*.R" | xargs grep read_csv`.
#### a) What are the programs?
#### b) Explain what this command is doing, part by part.

In [ ]:
# A) Programs: find, xargs, grep.
# B) find — program to traverse the filesystem.
# . — start at the current directory (recurse).
# -iname "*.R" — case‑insensitive name match for files like *.R (quotes prevent the shell from expanding *).
# | — pipe: send find’s results to the next program’s stdin.
# xargs — read file paths from stdin and build command lines.
# grep read_csv — for each batch of files from xargs, search for the string read_csv; print matching lines (typically with filenames when multiple files).

## Question 6
#### Install Docker on your machine. See [here](https://github.com/rjenki/BIOS512/blob/main/lecture18/docker_install.md) for instructions. 
#### a) Show the response when you run `docker run hello-world`.
#### b) Access Rstudio through a Docker container. Set your password and make sure your files show up on the Rstudio server. Type the command and the output you get below.
#### c) How do you log in to the RStudio server?

In [ ]:
# a) idk how to show this because i cant tell how to screenshot and put it in here but this is what my docker responded with in my powershell terminal:
PS C:\Users\Sara> docker run hello-world
Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
17eec7bbc9d7: Pull complete
Digest: sha256:f7931603f70e13dbd844253370742c4fc4202d290c80442b2e68706d8f33ce26
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/
# b) 
PS C:\Users\Sara> docker run -d --rm `
>>   -p 8787:8787 `
>>   -e PASSWORD="************" `
>>   -v "${PWD}:/home/rstudio/work" `
>>   --name rstudio `
>>   rocker/rstudio
f933e43709c5ee3edf067797086c8b0a10980cbff7216deda1a771e555cb9051
# c) Open a browser and go to: http://localhost:8787 username: rstudio password: ************
